In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openpyxl
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import scale

# Set working directory
os.chdir(".")
print(os.getcwd())

# Install and import necessary libraries
!pip install openpyxl
!pip install xgboost

import openpyxl
import xgboost as xgb

# Load the data
datal = pd.read_excel("dataforR.xlsx")

# Display the data
print(datal)

# Summary statistics
print(datal.iloc[:, 0:6].describe())

# Boxplot
datal.iloc[:, 0:6].boxplot()

# Checking for outliers
outliers_price = np.where(np.abs(scale(datal["LettucePrice"])) > 2)
outliers_prod = np.where(np.abs(scale(datal["Production"])) > 2)
outliers_sun = np.where(np.abs(scale(datal["Sunshine"])) > 2)
outliers_sol = np.where(np.abs(scale(datal["Solar.radiation"])) > 2)
outliers_pre = np.where(np.abs(scale(datal["Precipitation"])) > 2)

# Replace outliers with median value
datal.loc[outliers_price, "LettucePrice"] = np.median(datal["LettucePrice"])
datal.loc[outliers_prod, "Production"] = np.median(datal["Production"])
datal.loc[outliers_sun, "Sunshine"] = np.median(datal["Sunshine"])
datal.loc[outliers_sol, "Solar.radiation"] = np.median(datal["Solar.radiation"])
datal.loc[outliers_pre, "Precipitation"] = np.median(datal["Precipitation"])

# Divide data into training and testing sets
train_size = int(0.95 * len(datal))
train_data = datal.iloc[:train_size, :]
test_data = datal.iloc[train_size:, :]

# POLYNOMIAL REGRESSION
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Create a polynomial regression model
poly_reg = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())

# Fit the model to the training data
X_train = train_data.drop(columns=["Temperature", "SolarRadiation", "LettucePrice"])
y_train = train_data["LettucePrice"]
poly_reg.fit(X_train, y_train)

# Prediction on test data
X_test = test_data.drop(columns=["Temperature", "SolarRadiation", "LettucePrice"])
lm_prediction = poly_reg.predict(X_test)

# Results
result = pd.DataFrame({"OriginalP": test_data["LettucePrice"].values})
result["PredictedP"] = lm_prediction
result["Deviation"] = result["OriginalP"] - result["PredictedP"]
print(result)

# Calculate MSE and R-squared for the polynomial regression model
lm_mse = mean_squared_error(result["OriginalP"], result["PredictedP"])
lm_r2 = r2_score(result["OriginalP"], result["PredictedP"])

# Plotting using Matplotlib
plt.plot(result.index, result["OriginalP"], label="Original Price", color="blue")
plt.plot(result.index, result["PredictedP"], label="Predicted Price", color="red")
plt.scatter(result.index, result["OriginalP"] + result["Deviation"], label="Deviation", color="green", s=30)
plt.xlabel("Index")
plt.ylabel("Price")
plt.legend()
plt.show()

# XGBOOST
import xgboost as xgb

# Preparing training and testing set
train_data_xgb = train_data.drop(columns=["Temperature", "Sunshine"])
test_data_xgb = test_data.drop(columns=["Temperature", "Sunshine"])

x_train = train_data_xgb.iloc[:, 1:]
y_train = train_data_xgb["LettucePrice"].values

x_test = test_data_xgb.iloc[:, 1:].values

# Training XGBOOST model
param_grid = {
    "nrounds": [100, 200, 500],
    "max_depth": [3, 6, 9],
    "eta": [0.01, 0.1, 0.3]
}

dtrain = xgb.DMatrix(x_train, label=y_train)

cv_result = xgb.cv(
    params={
        "objective": "reg:squarederror",
        "eta": 0.1,
        "max_depth": 3
    },
    dtrain=dtrain,
    num_boost_round=1000,
    nfold=5,
    metrics="rmse",
    early_stopping_rounds=10,
    verbose_eval=False
)

best_iteration = cv_result["test-rmse-mean"].idxmin()
best_params = {
    "objective": "reg:squarederror",
    "max_depth": 3,
    "eta": 0.1
}

xgb_model = xgb.train(best_params, dtrain, num_boost_round=best_iteration)

# Predicting on test data
xgb_prediction = xgb_model.predict(xgb.DMatrix(x_test))

# Results
xgb_result = pd.DataFrame({"OriginalP": test_data_xgb["LettucePrice"].values})
xgb_result["PredictedP"] = xgb_prediction
xgb_result["Deviation"] = xgb_result["OriginalP"] - xgb_result["PredictedP"]
print(xgb_result)

# Calculate MSE and R-squared for XGBoost model
xgb_mse = mean_squared_error(xgb_result["OriginalP"], xgb_result["PredictedP"])
xgb_r2 = r2_score(xgb_result["OriginalP"], xgb_result["PredictedP"])

# Plotting using Matplotlib
plt.plot(xgb_result.index, xgb_result["OriginalP"], label="Original Price", color="blue")
plt.plot(xgb_result.index, xgb_result["PredictedP"], label="Predicted Price", color="red")
plt.scatter(xgb_result.index, xgb_result["OriginalP"] + xgb_result["Deviation"], label="Deviation", color="green", s=30)
plt.xlabel("Index")
plt.ylabel("Price")
plt.legend()
plt.show()

# REAL DATA PREDICTION
datap = pd.read_excel("prediction.xlsx")
datapMIN = pd.read_excel("predictionMIN.xlsx")
datapMAX = pd.read_excel("predictionMAX.xlsx")

# Linear regression prediction
lm_model_real = LinearRegression()
X_train_real = datal.drop(columns=["Temperature", "SolarRadiation", "LettucePrice"])
y_train_real = datal["LettucePrice"]
lm_model_real.fit(X_train_real, y_train_real)

lm_prediction_real_avg = lm_model_real.predict(datap)
lm_prediction_real_min = lm_model_real.predict(datapMIN)
lm_prediction_real_max = lm_model_real.predict(datapMAX)

# Create a dataframe with the time periods and predicted values
results = pd.DataFrame({"TimePeriod": range(1, len(lm_prediction_real_avg) + 1), "PredictedAvg": lm_prediction_real_avg})
results["PredictedMin"] = lm_prediction_real_min
results["PredictedMax"] = lm_prediction_real_max
print(results)

# Save results to an Excel file
results.to_excel("results.xlsx", index=False)

# PLOTTING using Matplotlib

# Define the start and end dates for your time period
start_date = pd.to_datetime("2023-06-01")
end_date = pd.to_datetime("2024-05-01")

# Create a sequence of dates using the start and end dates
date_sequence = pd.date_range(start_date, end_date, freq="M")

# Convert the date sequence to year-month format
year_month_sequence = date_sequence.strftime("%Y-%m")

# Define custom colors
avg_color = "#0072B2"    # Blue
min_color = "#D55E00"    # Orange
max_color = "#009E73"    # Green

# Create a plot using Matplotlib
plt.figure(figsize=(12, 6))
plt.fill_between(results["TimePeriod"], results["PredictedMin"], results["PredictedMax"], color="#F2F2F2")
plt.plot(results["TimePeriod"], results["PredictedAvg"], label="Average", color=avg_color)
plt.plot(results["TimePeriod"], results["PredictedMin"], label="Minimum", linestyle="--", color=min_color)
plt.plot(results["TimePeriod"], results["PredictedMax"], label="Maximum", linestyle="--", color=max_color)
plt.xlabel("Time Period")
plt.ylabel("Value")
plt.title("Average, Minimum, and Maximum Values Over Time")
plt.xticks(range(1, len(year_month_sequence) + 1), year_month_sequence, rotation=45)
plt.legend()
plt.grid(True)
plt.show()

# XGBOOST prediction
xgb_data = datal.drop(columns=["Temperature", "SolarRadiation", "LettucePrice"])
xgb_data_pred = datap.drop(columns=["Temperature", "SolarRadiation"])
xgb_data_pred_min = datapMIN.drop(columns=["Temperature", "SolarRadiation"])
xgb_data_pred_max = datapMAX.drop(columns=["Temperature", "SolarRadiation"])

# Training XGBOOST model
param_grid = {
    "nrounds": [100, 200, 500, 1000],
    "max_depth": [3, 6, 9, 12],
    "eta": [0.01, 0.1, 0.3, 0.5]
}

dtrain = xgb.DMatrix(xgb_data, label=y_train)

cv_result = xgb.cv(
    params={
        "objective": "reg:squarederror",
        "eta": 0.1,
        "max_depth": 3
    },
    dtrain=dtrain,
    num_boost_round=1000,
    nfold=5,
    metrics="rmse",
    early_stopping_rounds=10,
    verbose_eval=False
)

best_iteration = cv_result["test-rmse-mean"].idxmin()
best_params = {
    "objective": "reg:squarederror",
    "max_depth": 3,
    "eta": 0.1
}

xgb_model_real = xgb.train(best_params, dtrain, num_boost_round=best_iteration)

# Predicting on test data
xgb_prediction_real = xgb_model_real.predict(xgb.DMatrix(xgb_data_pred))
xgb_prediction_real_min = xgb_model_real.predict(xgb.DMatrix(xgb_data_pred_min))
xgb_prediction_real_max = xgb_model_real.predict(xgb.DMatrix(xgb_data_pred_max))

# Create a dataframe with the time periods and predicted values
xgb_results = pd.DataFrame({"TimePeriod": range(1, len(xgb_prediction_real) + 1), "PredictedAvg": xgb_prediction_real})
xgb_results["PredictedMin"] = xgb_prediction_real_min
xgb_results["PredictedMax"] = xgb_prediction_real_max
print(xgb_results)

# Create the plot
plt.figure(figsize=(12, 6))
plt.plot(xgb_results["TimePeriod"], xgb_results["PredictedAvg"], label="Predicted Average", color="blue")
plt.plot(xgb_results["TimePeriod"], xgb_results["PredictedMin"], label="Predicted Min", color="red", linestyle="--")
plt.plot(xgb_results["TimePeriod"], xgb_results["PredictedMax"], label="Predicted Max", color="green", linestyle="--")
plt.xlabel("Time Period")
plt.ylabel("Predicted Value")
plt.title("Predicted Values Over Time")
plt.legend()
plt.grid(True)
plt.show()